## Import Libraries

In [ ]:
%%capture
!pip install nltk



In [ ]:
%%capture

!pip install emoji

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import spacy
import nltk
import string
from nltk.tokenize import word_tokenize
import re
import gensim
import matplotlib.pyplot as plt
# import pyarabic.araby as araby
import sqlite3
import emoji
#import stanza
#import camel_tools as ct
from nltk.corpus import stopwords
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)
# from tnkeeh import Tnkeeh
from sklearn.utils.class_weight import compute_class_weight
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.models import Sequential
from keras.layers import LSTM, Dense, SpatialDropout1D, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding

In [ ]:
import sqlite3
import pandas as pd
import re
import emoji
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
X_train_path = 'X_train.csv'
X_test_path = 'X_test.csv'
y_train_path = 'y_train.csv'
y_test_path = 'y_test.csv'

X_train = pd.read_csv(X_train_path,encoding='utf-8')
X_test = pd.read_csv(X_test_path,encoding='utf-8')
y_train = pd.read_csv(y_train_path,encoding='utf-8')
y_test = pd.read_csv(y_test_path,encoding='utf-8')


# Preprocessing


In [ ]:
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()


In [ ]:
# Handle NaN values in text columns by dropping them along with corresponding labels
X_train = X_train.dropna(subset=['text'])
X_test = X_test.dropna(subset=['text'])
y_train = y_train[X_train.index]
y_test = y_test[X_test.index]

## class weight computation

This function computes class weights based on the balanced class distribution in the training data.

### Parameters:
- `classes`: numpy array containing all unique class labels in both training and testing data.
- `y_train`: numpy array containing class labels for the training data.
- `y_test`: numpy array containing class labels for the testing data.

### Returns:
- `class_weight_dict`: dictionary containing class weights computed based on the balanced class distribution.


In [ ]:
classes = np.unique(np.concatenate((y_train, y_test)))
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = {cls: weight for cls, weight in zip(classes, class_weights)}

# Logistic Regression with TF-IDF Vectorization
1. **Vectorize Text Data using TF-IDF:**
   - Create a TF-IDF vectorizer.
   - Fit the vectorizer on the training text data (`X_train['text']`) to learn the vocabulary and compute IDF values.
   - Transform both the training and testing text data into TF-IDF features.

2. **Train Logistic Regression Model with Class Weights:**
   - Instantiate a Logistic Regression model with specified class weights (`class_weight_dict`) to handle class imbalance.
   - Fit the Logistic Regression model on the TF-IDF features of the training data.

3. **Predict on the Test Set:**
   - Use the trained Logistic Regression model to predict labels for the TF-IDF features of the testing data (`X_test_tfidf`).

4. **Calculate Accuracy:**
   - Compute the accuracy score of the Logistic Regression model on the test set to evaluate its performance.

### Parameters:
- `X_train['text']`: Training text data containing sentences/documents.
- `X_test['text']`: Testing text data containing sentences/documents.
- `y_train`: Labels corresponding to the training text data.
- `y_test`: Labels corresponding to the testing text data.
- `class_weight_dict`: Dictionary containing class weights for balancing class distribution.

### Returns:
- `accuracy_tfidf`: Accuracy score of the Logistic Regression model on the test set.



In [ ]:
# Vectorize text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['text'])
X_test_tfidf = vectorizer.transform(X_test['text'])

In [ ]:
print("X_train_tfidf shape:", X_train_tfidf.shape)
print("X_test_tfidf shape:", X_test_tfidf.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Train the Logistic Regression model with class weights
lr_model_tfidf = LogisticRegression(class_weight=class_weight_dict, max_iter=1000)
lr_model_tfidf.fit(X_train_tfidf, y_train)

# Predict on the test set
lr_labels_tfidf = lr_model_tfidf.predict(X_test_tfidf)

# Calculate accuracy
accuracy_tfidf = accuracy_score(y_test, lr_labels_tfidf)
print(f'Test Accuracy with TF-IDF and Logistic Regression: {accuracy_tfidf:.4f}')

X_train_tfidf shape: (117972, 179933)
X_test_tfidf shape: (29482, 179933)
y_train shape: (117972,)
y_test shape: (29482,)
Test Accuracy with TF-IDF and Logistic Regression: 0.8016


In [ ]:
from sklearn.metrics import f1_score
macro_f1 = f1_score(y_test, lr_labels_tfidf, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.7696901140061857


In [ ]:
accuracy = accuracy_score(y_test, lr_labels_tfidf)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.8016


In [ ]:
print(classification_report(y_test, lr_labels_tfidf))

              precision    recall  f1-score   support

          EG       0.89      0.82      0.85     11459
          LB       0.82      0.83      0.83      5570
          LY       0.78      0.80      0.79      7256
          MA       0.69      0.74      0.71      2276
          SD       0.63      0.71      0.67      2921

    accuracy                           0.80     29482
   macro avg       0.76      0.78      0.77     29482
weighted avg       0.81      0.80      0.80     29482



In [ ]:
import joblib
joblib.dump(lr_model_tfidf, 'lr_pipeline_tfidf.joblib')

['lr_pipeline_tfidf.joblib']

# Logistic Regression with CountVectorizer
1. **Vectorize Text Data using CountVectorizer:**
   - Create a CountVectorizer object.
   - Fit the vectorizer on the training text data (`X_train['text']`) to learn the vocabulary.
   - Transform both the training and testing text data into count-based features.

2. **Train Logistic Regression Model with Class Weights:**
   - Instantiate a Logistic Regression model with specified class weights (`class_weight_dict`) to handle class imbalance.
   - Fit the Logistic Regression model on the count-based features of the training data.

3. **Predict on the Test Set:**
   - Use the trained Logistic Regression model to predict labels for the count-based features of the testing data (`X_test_count`).

4. **Calculate Accuracy:**
   - Compute the accuracy score of the Logistic Regression model on the test set to evaluate its performance.

### Parameters:
- `X_train['text']`: Training text data containing sentences/documents.
- `X_test['text']`: Testing text data containing sentences/documents.
- `y_train`: Labels corresponding to the training text data.
- `y_test`: Labels corresponding to the testing text data.
- `class_weight_dict`: Dictionary containing class weights for balancing class distribution.

### Returns:
- `accuracy_count`: Accuracy score of the Logistic Regression model on the test set.



In [ ]:
# Vectorize text data using CountVectorizer
vectorizer = CountVectorizer()
X_train_count = vectorizer.fit_transform(X_train['text'])
X_test_count = vectorizer.transform(X_test['text'])

# Train the Logistic Regression model with class weights
lr_model_count = LogisticRegression(class_weight=class_weight_dict, max_iter=1000)
lr_model_count.fit(X_train_count, y_train)

# Predict on the test set
lr_labels_count = lr_model_count.predict(X_test_count)

# Calculate accuracy
accuracy_count = accuracy_score(y_test, lr_labels_count)
print(f'Test Accuracy with CountVectorizer and Logistic Regression: {accuracy_count:.4f}')


Test Accuracy with CountVectorizer and Logistic Regression: 0.8011


In [ ]:
from sklearn.metrics import f1_score
macro_f1 = f1_score(y_test, lr_labels_count, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.7666504122916823


In [ ]:
accuracy = accuracy_score(y_test, lr_labels_count)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.8011


In [ ]:
print(classification_report(y_test, lr_labels_count))

              precision    recall  f1-score   support

          EG       0.88      0.83      0.85     11459
          LB       0.81      0.84      0.82      5570
          LY       0.78      0.80      0.79      7256
          MA       0.70      0.71      0.71      2276
          SD       0.64      0.68      0.66      2921

    accuracy                           0.80     29482
   macro avg       0.76      0.77      0.77     29482
weighted avg       0.80      0.80      0.80     29482



In [ ]:
joblib.dump(lr_model_count, 'lr_pipeline_count.joblib')

['lr_pipeline_count.joblib']

# Multinomial Naive Bayes with TF-IDF Vectorization
1. **Compute Class Weights:**
   - Calculate class weights using `compute_class_weight` based on the distribution of classes in the training data.

2. **Set Class Priors:**
   - Normalize the class weights to get class priors by dividing each class weight by the sum of all class weights.

3. **Instantiate Multinomial Naive Bayes Model:**
   - Create a Multinomial Naive Bayes model (`nb_model_tfidf`) with class priors set to the normalized class weights.

4. **Fit the Model:**
   - Train the Multinomial Naive Bayes model on the TF-IDF vectorized training data (`X_train_tfidf`) and corresponding labels (`y_train`).

5. **Predict on the Test Set:**
   - Use the trained model to predict labels for the TF-IDF vectorized testing data (`X_test_tfidf`).

### Parameters:
- `X_train_tfidf`: TF-IDF vectorized training data.
- `X_test_tfidf`: TF-IDF vectorized testing data.
- `y_train`: Labels corresponding to the training data.
- `class_weights`: Class weights computed using `compute_class_weight`.
- `class_prior`: Class priors normalized from class weights.

### Returns:
- `nb_labels_tfidf`: Predicted labels for the TF-IDF vectorized testing data.


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

class_prior = class_weights / class_weights.sum()
nb_model_tfidf = MultinomialNB(class_prior=class_prior)

nb_model_tfidf.fit(X_train_tfidf, y_train)
nb_labels_tfidf = nb_model_tfidf.predict(X_test_tfidf)

In [ ]:
from sklearn.metrics import f1_score
nb_macro_f1_tfidf = f1_score(y_test, nb_labels_tfidf, average='macro')
print("Macro F1-score:", nb_macro_f1_tfidf)

Macro F1-score: 0.7406963220528369


In [ ]:
nb_accuracy_tfidf = accuracy_score(y_test, nb_labels_tfidf)
print(f'Accuracy: {nb_accuracy_tfidf:.4f}')

Accuracy: 0.7835


In [ ]:
print(classification_report(y_test, nb_labels_tfidf))

              precision    recall  f1-score   support

          EG       0.88      0.83      0.85     11459
          LB       0.86      0.83      0.84      5570
          LY       0.89      0.70      0.79      7256
          MA       0.50      0.79      0.61      2276
          SD       0.53      0.72      0.61      2921

    accuracy                           0.78     29482
   macro avg       0.73      0.77      0.74     29482
weighted avg       0.81      0.78      0.79     29482



In [ ]:
joblib.dump(nb_model_tfidf, 'nb_pipeline_tfidf.joblib')

['nb_pipeline_tfidf.joblib']

# Multinomial Naive Bayes with CountVectorizer

1. **Compute Class Weights:**
   - Calculate class weights using `compute_class_weight` based on the distribution of classes in the training data.

2. **Set Class Priors:**
   - Normalize the class weights to get class priors by dividing each class weight by the sum of all class weights.

3. **Instantiate Multinomial Naive Bayes Model:**
   - Create a Multinomial Naive Bayes model (`nb_model_count`) with class priors set to the normalized class weights.

4. **Fit the Model:**
   - Train the Multinomial Naive Bayes model on the CountVectorizer vectorized training data (`X_train_count`) and corresponding labels (`y_train`).

5. **Predict on the Test Set:**
   - Use the trained model to predict labels for the CountVectorizer vectorized testing data (`X_test_count`).

### Parameters:
- `X_train_count`: CountVectorizer vectorized training data.
- `X_test_count`: CountVectorizer vectorized testing data.
- `y_train`: Labels corresponding to the training data.
- `class_weights`: Class weights computed using `compute_class_weight`.
- `class_prior`: Class priors normalized from class weights.

### Returns:
- `nb_labels_count`: Predicted labels for the CountVectorizer vectorized testing data.


In [22]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression


class_prior = class_weights / class_weights.sum()
nb_model_count = MultinomialNB(class_prior=class_prior)


nb_model_count.fit(X_train_count, y_train)
nb_labels_count = nb_model_count.predict(X_test_count)

In [ ]:
from sklearn.metrics import f1_score
nb_macro_f1 = f1_score(y_test, nb_labels_count, average='macro')
print("Macro F1-score:", nb_macro_f1)

Macro F1-score: 0.7849156085676153


In [ ]:
nb_accuracy = accuracy_score(y_test, nb_labels_count)
print(f'Accuracy: {nb_accuracy:.4f}')

Accuracy: 0.8242


In [ ]:
print(classification_report(y_test, nb_labels_count))

              precision    recall  f1-score   support

          EG       0.86      0.90      0.88     11459
          LB       0.86      0.85      0.86      5570
          LY       0.85      0.78      0.81      7256
          MA       0.70      0.72      0.71      2276
          SD       0.66      0.67      0.67      2921

    accuracy                           0.82     29482
   macro avg       0.79      0.78      0.78     29482
weighted avg       0.82      0.82      0.82     29482



In [29]:
joblib.dump(nb_model_count, 'nb_pipeline_count.joblib')

['nb_pipeline_count.joblib']

# Testing the model


In [31]:
model = joblib.load('nb_pipeline_count.joblib')  # Load the model using joblib


In [34]:
from sklearn.feature_extraction.text import CountVectorizer

# Define the new sentence
new_sentence = "مبروك العواشر وتعيد وتعاود:"

# Preprocess the new sentence using the same vectorizer used for training
new_sentence_vec = vectorizer.transform([new_sentence])

# Predict the label for the new sentence using the trained logistic regression model
predicted_label = model.predict(new_sentence_vec)

print("Predicted label:", predicted_label)


Predicted label: ['MA']


# deep learning



1. **Label Encoding:**
   - Encode the target variable (`y_train` and `y_test`) using `LabelEncoder`.
   - Transform the original class labels into numerical values.

2. **One-Hot Encoding (Optional):**
   - Convert the encoded labels into categorical one-hot encoded format using `to_categorical` if needed.

3. **Compute Class Weights:**
   - Calculate class weights using `compute_class_weight` to handle imbalanced class distributions.
   - Pass the encoded class labels (`y_train_encoded`) to compute class weights.

### Parameters:
- `y_train`: Target variable (labels) for the training data.
- `y_test`: Target variable (labels) for the testing data.
- `label_encoder`: Instance of the `LabelEncoder` class used for label encoding.
- `y_train_encoded`: Encoded labels for the training data.
- `y_test_encoded`: Encoded labels for the testing data.
- `y_train_categorical`: One-hot encoded labels for the training data (if applicable).
- `y_test_categorical`: One-hot encoded labels for the testing data (if applicable).
- `class_weight_dict`: Dictionary containing the computed class weights.

### Returns:
- `class_weight_dict`: Dictionary containing class weights for each class.



In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, GRU, SimpleRNN
from keras.optimizers import Adam
from tensorflow.keras.regularizers import l2, l1

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)

Class weights: {0: 0.5120645875382512, 1: 1.0716933139534883, 2: 0.8083595998355488, 3: 2.5540593201991775, 4: 2.0601065223085655}


## Tokenization and Padding

The provided code initializes a Tokenizer object to tokenize the text data and pad sequences to ensure uniform length for modeling.

- `MAX_NB_WORDS`: Maximum number of unique words to tokenize (set to 50,000).
- `MAX_SEQUENCE_LENGTH`: Maximum length of sequences after padding (set to 60).

## Model Architecture

The defined model architecture is a recurrent neural network (RNN) with an embedding layer, spatial dropout, LSTM layer, and dense layer.

- Embedding Layer: Maps the tokenized words to dense vectors of fixed size (100-dimensional).
- Spatial Dropout: Applies dropout along the spatial dimensions to prevent overfitting (dropout rate set to 0.5).
- LSTM Layer: Long Short-Term Memory layer with 50 units and dropout of 0.5 for input and recurrent connections.
- Dense Layer: Output layer with softmax activation function for multiclass classification.

## Model Compilation and Training

The model is compiled using categorical cross-entropy loss and the Adam optimizer.

- Early Stopping: Stops training if the validation loss does not improve for 3 epochs to prevent overfitting.

## Training Process

The model is trained on the padded sequences of the training data with class weights to handle class imbalance.

- Training occurs over 10 epochs with a batch size of 300 and a validation split of 0.2.

## Model Evaluation

The trained model is evaluated on the test set, and the test accuracy is printed.


In [ ]:
# Tokenizer initialization
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 60
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(X_train['text'])

# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train['text'])
X_test_seq = tokenizer.texts_to_sequences(X_test['text'])

# Pad sequences to the same length
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH)

# Encode the labels as one-hot vectors
y_train_categorical = to_categorical([classes.tolist().index(label) for label in y_train], num_classes=len(classes))
y_test_categorical = to_categorical([classes.tolist().index(label) for label in y_test], num_classes=len(classes))

# Define the model with reduced complexity and increased dropout
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, 100, input_length=MAX_SEQUENCE_LENGTH))  # Embedding dimension
model.add(SpatialDropout1D(0.5))  # Increased dropout rate
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))  # LSTM units with dropout
model.add(Dense(len(classes), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with class weights
history = model.fit(X_train_pad, y_train_categorical, epochs=10, batch_size=300, validation_split=0.2, callbacks=[early_stop], class_weight=class_weight_dict)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_pad, y_test_categorical)
print(f'Test Accuracy: {accuracy:.4f}')


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 60, 100)           5000000   
                                                                 
 spatial_dropout1d_2 (Spati  (None, 60, 100)           0         
 alDropout1D)                                                    
                                                                 
 lstm_2 (LSTM)               (None, 50)                30200     
                                                                 
 dense_2 (Dense)             (None, 5)                 255       
                                                                 
Total params: 5030455 (19.19 MB)
Trainable params: 5030455 (19.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
315/315 [==============================] - 101s 310ms/step - los

## Confusion Matrix and Classification Report

This code snippet demonstrates the generation of a confusion matrix and a classification report to evaluate the performance of a classification model.

### Steps:
1. **Predict Probabilities:**
   - Use the trained model to predict probabilities for each class for the test data (`X_test_pad`).

2. **Convert Probabilities to Class Labels:**
   - Convert the predicted probabilities into class labels by selecting the class with the highest probability.

3. **Compute Confusion Matrix:**
   - Generate the confusion matrix using the true class labels (`y_test_categorical`) and the predicted class labels (`y_pred`).

4. **Print Confusion Matrix:**
   - Print the confusion matrix to visualize the performance of the classification model.

5. **Calculate Precision, Recall, and F1-score:**
   - Compute precision, recall, and F1-score using the `classification_report` function.

### Parameters:
- `X_test_pad`: Test data with padded sequences (features).
- `y_test_categorical`: True labels of the test data in one-hot encoded format.
- `model`: Trained classification model.

### Returns:
- `conf_matrix`: Confusion matrix showing the number of true positive, true negative, false positive, and false negative predictions for each class.
- `report`: Classification report containing precision, recall, and F1-score for each class, along with the overall accuracy, weighted average precision, recall, and F1-score.



In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Predict probabilities for each class for the test data
y_pred_prob = model.predict(X_test_pad)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(np.argmax(y_test_categorical, axis=1), y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Calculate precision, recall, and F1-score
report = classification_report(np.argmax(y_test_categorical, axis=1), y_pred)
print("Classification Report:")
print(report)

924/924 [==============================] - 24s 25ms/step
Confusion Matrix:
[[9649  315  457  191  872]
 [ 243 4602  278  139  316]
 [ 530  370 5535  289  544]
 [ 190   96  181 1659  157]
 [ 336  113  194   89 2200]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.84      0.86     11484
           1       0.84      0.83      0.83      5578
           2       0.83      0.76      0.80      7268
           3       0.70      0.73      0.71      2283
           4       0.54      0.75      0.63      2932

    accuracy                           0.80     29545
   macro avg       0.76      0.78      0.77     29545
weighted avg       0.81      0.80      0.80     29545



In [ ]:
model.save("lstm_model_2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# compare the confusion matrices for each model:

### Model Comparison: Confusion Matrix

| Model | Precision (EG) | Recall (EG) | F1-Score (EG) | Precision (LB) | Recall (LB) | F1-Score (LB) | Precision (LY) | Recall (LY) | F1-Score (LY) | Precision (MA) | Recall (MA) | F1-Score (MA) | Precision (SD) | Recall (SD) | F1-Score (SD) | Accuracy |
|-------|----------------|-------------|----------------|----------------|-------------|----------------|----------------|-------------|----------------|----------------|-------------|----------------|----------------|-------------|----------------|----------|
| Logistic Regression with TF-IDF Vectorization | 0.89 | 0.82 | 0.85 | 0.82 | 0.83 | 0.83 | 0.78 | 0.80 | 0.79 | 0.69 | 0.74 | 0.71 | 0.63 | 0.71 | 0.67 | 0.80 |
| Logistic Regression with CountVectorizer | 0.88 | 0.83 | 0.85 | 0.81 | 0.84 | 0.82 | 0.78 | 0.80 | 0.79 | 0.70 | 0.71 | 0.71 | 0.64 | 0.68 | 0.66 | 0.80 |
| Multinomial Naive Bayes with TF-IDF Vectorization | 0.88 | 0.83 | 0.85 | 0.86 | 0.83 | 0.84 | 0.89 | 0.70 | 0.79 | 0.50 | 0.79 | 0.61 | 0.53 | 0.72 | 0.61 | 0.78 |
| Multinomial Naive Bayes with CountVectorizer | 0.86 | 0.90 | 0.88 | 0.86 | 0.85 | 0.86 | 0.85 | 0.78 | 0.81 | 0.70 | 0.72 | 0.71 | 0.66 | 0.67 | 0.67 | 0.82 |
| Deep Learning Model | 0.88 | 0.84 | 0.86 | 0.84 | 0.83 | 0.83 | 0.83 | 0.76 | 0.80 | 0.70 | 0.73 | 0.71 | 0.54 | 0.75 | 0.63 | 0.80 |

### Interpretation:
- The table provides precision, recall, and F1-score for each class (dialect) for all models.
- The accuracy of each model is also included in the last column.
- Based on these metrics, you can compare the performance of different models in classifying Arabic dialects.

This comparison allows for a quick evaluation of each model's performance across different evaluation metrics.

# interpret the results and determine the best model, let's analyze the metrics provided in the table:

1. **Precision, Recall, and F1-Score:**
   - **Precision:** It measures the accuracy of the positive predictions. A high precision indicates a low false positive rate, meaning that the model is good at predicting positive cases correctly.
   - **Recall:** It measures the ability of the model to capture all positive instances. A high recall indicates a low false negative rate, meaning that the model is good at identifying all positive cases.
   - **F1-Score:** It is the harmonic mean of precision and recall. It provides a balance between precision and recall.

2. **Accuracy:**
   - It measures the overall correctness of the model's predictions across all classes.

Based on the provided results:
- **Logistic Regression with CountVectorizer** and **Multinomial Naive Bayes with CountVectorizer** achieve similar performance across all metrics, with slightly higher accuracy compared to the other models.
- **Deep Learning Model** performs well in terms of accuracy but has slightly lower precision, recall, and F1-score compared to the other models for certain classes.
- **Logistic Regression with TF-IDF Vectorization** shows good performance but may have slightly lower accuracy and F1-scores compared to the CountVectorizer-based models.

**Choosing the Best Model:**
- The choice of the best model depends on the specific requirements and priorities of the task. If overall accuracy is the primary concern, the CountVectorizer-based models or the Deep Learning Model may be preferred.
- If balanced performance across precision, recall, and F1-score is crucial, the Logistic Regression or Naive Bayes models with CountVectorizer might be preferable.
- It's essential to consider factors such as computational complexity, interpretability, and deployment constraints when selecting the final model.